# Feels Protocol Parameter Sweep Analysis

This notebook analyzes the results from comprehensive parameter sweeps to identify optimal
fee allocation strategies for the Feels protocol launch.

## Objectives

1. Load and analyze parameter sweep results from batch runs
2. Compare performance across different fee structures and market conditions
3. Identify optimal fee allocation strategies
4. Generate governance-ready recommendations
5. Understand trade-offs between floor growth, protocol sustainability, and ecosystem incentives

## Prerequisites

Run parameter sweeps using:
```bash
just sweep --sweep-type full --hours 168
```

Or run individual sweeps:
```bash
just sweep --sweep-type fee_range
just sweep --sweep-type market_conditions
```

In [ ]:
# Import required libraries
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import json
from datetime import datetime
import glob

# Setup path handling for both local and online execution
try:
    # Try importing the package directly (works when installed via pip)
    import feels_sim
    print("Package imported successfully")
    project_root = None  # Not needed when package is installed
    
except ImportError:
    # Fallback for local development - add project root to path
    print("Package not installed, using local development setup...")
    project_root = Path().cwd().parent
    sys.path.insert(0, str(project_root))
    
    try:
        import feels_sim
        print("Local development imports successful")
    except ImportError as e:
        print(f"Import failed: {e}")
        print("Please ensure feels_sim package is installed or you're running from the project directory")
        raise

# Import centralized plotting functions
from feels_sim.plotting import (
    setup_plot_style, 
    DEFAULT_STYLE, 
    create_figure_with_style, 
    create_parameter_sweep_plots,
    apply_professional_styling,
    save_plot
)

# Setup consistent plotting style
setup_plot_style()

print("Libraries imported successfully with centralized styling")

## 1. Load Parameter Sweep Results

In [ ]:
# Load the most recent parameter sweep results
# Setup runs directory for both local and online execution
if project_root is not None:
    # Local development
    runs_dir = project_root / "experiments" / "runs"
else:
    # Online/installed package execution
    runs_dir = Path("experiments") / "runs"

runs_dir.mkdir(parents=True, exist_ok=True)

# Find the most recent results files
all_results_files = list(runs_dir.glob("all_results_*.json"))
summary_files = list(runs_dir.glob("comparative_summary_*.json"))
fee_sweep_files = list(runs_dir.glob("fee_sweep_*.json"))
market_sweep_files = list(runs_dir.glob("market_sweep_*.json"))

print(f"Found {len(all_results_files)} complete result files")
print(f"Found {len(summary_files)} summary files")
print(f"Found {len(fee_sweep_files)} fee sweep files")
print(f"Found {len(market_sweep_files)} market sweep files")

# Check if we have sufficient data for comprehensive analysis
sufficient_data = False
if all_results_files:
    latest_results_file = max(all_results_files, key=lambda x: x.stat().st_mtime)
    with open(latest_results_file, 'r') as f:
        all_results = json.load(f)
    
    successful_results = [r for r in all_results if r.get('success', False)]
    
    # Check comprehensive data requirements for full analysis
    scenarios = set(r['scenario_name'] for r in successful_results)
    fee_levels = set(r.get('base_fee_bps', 30) for r in successful_results)
    market_conditions = set(r.get('market_condition', 'normal') for r in successful_results)
    
    # Comprehensive criteria for full analysis including fee sensitivity
    sufficient_data = (
        len(successful_results) >= 20 and    # Need many results for robust analysis
        len(scenarios) >= 4 and              # Need good scenario coverage
        len(fee_levels) >= 3 and             # Need multiple fee levels for elasticity analysis
        len(market_conditions) >= 2          # Need multiple market conditions
    )
    
    print(f"Loaded {len(all_results)} simulation results")
    print(f"{len(successful_results)} successful simulations, {len(all_results) - len(successful_results)} failed")
    print(f"Scenarios found: {list(scenarios)} ({len(scenarios)} unique)")
    print(f"Fee levels found: {sorted(fee_levels)} ({len(fee_levels)} unique)")
    print(f"Market conditions found: {sorted(market_conditions)} ({len(market_conditions)} unique)")
    print(f"")
    print(f"Data sufficiency check:")
    print(f"   Results: {len(successful_results)} >= 20: {len(successful_results) >= 20}")
    print(f"   Scenarios: {len(scenarios)} >= 4: {len(scenarios) >= 4}")
    print(f"   Fee levels: {len(fee_levels)} >= 3: {len(fee_levels) >= 3}")
    print(f"   Market conditions: {len(market_conditions)} >= 2: {len(market_conditions) >= 2}")
    print(f"  → Comprehensive analysis ready: {sufficient_data}")

if not sufficient_data:
    print("\nRUNNING AUTOMATIC PARAMETER SWEEP")
    print("Insufficient data for comprehensive analysis - generating full parameter sweep...")
    print("This enables:")
    print("  • Fee elasticity analysis with multiple fee levels per scenario")
    print("  • Market condition impact analysis")
    print("  • Robust statistical comparisons")
    print("  • Pareto frontier optimization")
    
    # Import simulation components
    if project_root is not None:
        sys.path.insert(0, str(project_root))
    
    from feels_sim.config import SimulationConfig
    from feels_sim.core import FeelsSimulation
    from feels_sim.metrics import calculate_key_metrics
    import time
    
    # Define comprehensive parameter sweep
    scenarios_to_test = [
        'default',
        'protocol_sustainable', 
        'creator_incentive',
        'balanced_growth',
        'minimum_protocol',
        'maximum_protocol'
    ]
    
    fee_levels_to_test = [20, 30, 40, 50]  # basis points
    market_conditions = [
        {'name': 'normal', 'volatility': 0.05, 'trend': 0.0},
        {'name': 'bull_market', 'volatility': 0.04, 'trend': 0.1},
        {'name': 'bear_market', 'volatility': 0.07, 'trend': -0.1},
        {'name': 'high_volatility', 'volatility': 0.12, 'trend': 0.0}
    ]
    
    total_combinations = len(scenarios_to_test) * len(fee_levels_to_test) * len(market_conditions)
    print(f"")
    print(f"Running comprehensive parameter sweep:")
    print(f"  • {len(scenarios_to_test)} scenarios × {len(fee_levels_to_test)} fee levels × {len(market_conditions)} market conditions")
    print(f"  • {total_combinations} total simulations")
    print(f"  • Estimated time: 8-15 minutes")
    print(f"  • This will enable all advanced analysis features")
    print("")
    
    results = []
    current = 0
    start_time = time.time()
    
    for scenario in scenarios_to_test:
        for fee_bps in fee_levels_to_test:
            for market in market_conditions:
                current += 1
                
                try:
                    print(f"[{current:2d}/{total_combinations}] {scenario:18s} @ {fee_bps:2d}bps in {market['name']:14s}...", end=" ")
                    
                    # Create configuration
                    config = SimulationConfig.create_fee_scenario(
                        scenario,
                        base_fee_bps=fee_bps,
                        sol_volatility_daily=market['volatility'],
                        sol_trend_bias=market['trend']
                    )
                    
                    # Run simulation (48 hours for balance of speed vs. accuracy)
                    sim = FeelsSimulation(config)
                    sim_results = sim.run(hours=48)
                    
                    # Calculate metrics
                    analysis = calculate_key_metrics(sim_results.snapshots)
                    
                    # Create result entry
                    result = {
                        'scenario_name': scenario,
                        'base_fee_bps': fee_bps,
                        'market_condition': market['name'],
                        'success': True,
                        'config': {
                            'protocol_fee_rate_bps': config.protocol_fee_rate_bps,
                            'creator_fee_rate_bps': config.creator_fee_rate_bps,
                            'base_fee_bps': config.base_fee_bps,
                            'sol_volatility_daily': config.sol_volatility_daily,
                            'sol_trend_bias': config.sol_trend_bias
                        },
                        'analysis': analysis,
                        'run_time_seconds': time.time() - start_time
                    }
                    results.append(result)
                    print("")
                    
                except Exception as e:
                    print(f"ERROR {str(e)[:30]}...")
                    result = {
                        'scenario_name': scenario,
                        'base_fee_bps': fee_bps,
                        'market_condition': market['name'],
                        'success': False,
                        'error': str(e),
                        'config': {},
                        'analysis': {},
                        'run_time_seconds': time.time() - start_time
                    }
                    results.append(result)
    
    # Save results
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = runs_dir / f"all_results_comprehensive_{timestamp}.json"
    
    with open(output_file, 'w') as f:
        json.dump(results, f, indent=2)
    
    # Create summary
    successful_results = [r for r in results if r.get('success', False)]
    failed_results = [r for r in results if not r.get('success', False)]
    
    summary = {
        "timestamp": timestamp,
        "total_simulations": len(results),
        "successful": len(successful_results),
        "failed": len(failed_results),
        "scenarios_tested": len(scenarios_to_test),
        "fee_levels_tested": fee_levels_to_test,
        "market_conditions_tested": [m['name'] for m in market_conditions],
        "run_time_minutes": (time.time() - start_time) / 60,
        "type": "comprehensive_parameter_sweep"
    }
    
    summary_file = runs_dir / f"comprehensive_summary_{timestamp}.json"
    with open(summary_file, 'w') as f:
        json.dump(summary, f, indent=2)
    
    print(f"\nCOMPREHENSIVE PARAMETER SWEEP COMPLETE!")
    print(f"Generated {len(successful_results)} successful simulations")
    print(f"Failed simulations: {len(failed_results)}")
    print(f"Results saved to: {output_file.name}")
    print(f"Runtime: {summary['run_time_minutes']:.1f} minutes")
    print(f"")
    print(f"ALL ADVANCED ANALYSIS FEATURES NOW ENABLED:")
    print(f"   Fee elasticity analysis with multiple fee levels")
    print(f"   Market condition impact comparison")
    print(f"   Robust Pareto frontier optimization")
    print(f"   Multi-objective parameter recommendations")
    
    # Update variables for rest of notebook
    all_results = results
    latest_results_file = output_file
    
else:
    print(f"COMPREHENSIVE DATA FOUND - All analysis features enabled")
    print(f"Loading existing results: {latest_results_file.name}")
    
    latest_summary_file = max(summary_files, key=lambda x: x.stat().st_mtime) if summary_files else None
    
    if latest_summary_file:
        with open(latest_summary_file, 'r') as f:
            summary_data = json.load(f)
        print(f"Loaded summary: {latest_summary_file.name}")
    else:
        summary_data = None

# Final data preparation
successful_results = [r for r in all_results if r.get('success', False)]
failed_results = [r for r in all_results if not r.get('success', False)]

print(f"\n=== FINAL DATA SUMMARY ===")
print(f"Total simulations: {len(all_results)}")
print(f"Successful: {len(successful_results)}")
print(f"Failed: {len(failed_results)}")

scenarios = sorted(set(r['scenario_name'] for r in successful_results))
fee_levels = sorted(set(r.get('base_fee_bps', 30) for r in successful_results))
market_conditions = sorted(set(r.get('market_condition', 'normal') for r in successful_results))

print(f"Scenarios ({len(scenarios)}): {scenarios}")
print(f"Fee levels ({len(fee_levels)}): {fee_levels}")
print(f"Market conditions ({len(market_conditions)}): {market_conditions}")
print(f"")
print("Ready for comprehensive analysis including:")
print("   • Fee elasticity analysis")
print("   • Market condition comparisons") 
print("   • Multi-objective optimization")
print("   • Robust statistical insights")

## 2. Convert Results to DataFrames for Analysis

In [ ]:
# Convert results to DataFrame for easier analysis
if 'successful_results' in locals() and successful_results:
    
    # Extract key metrics into a flat structure
    df_data = []
    
    for result in successful_results:
        config = result['config']
        analysis = result['analysis']
        
        # Handle both old and new fee parameter names
        if 'protocol_fee_rate_bps' in config:
            # New fee system (basis points)
            protocol_fee_bps = config['protocol_fee_rate_bps']
            creator_fee_bps = config['creator_fee_rate_bps']
            buffer_pct = 100.0 - (protocol_fee_bps + creator_fee_bps) / 100.0
            treasury_pct = protocol_fee_bps / 100.0
            creator_pct = creator_fee_bps / 100.0
        else:
            # Old fee system (percentages) - fallback
            treasury_pct = config.get('treasury_share_pct', 1.0)
            creator_pct = config.get('creator_share_pct', 0.5)
            buffer_pct = config.get('buffer_share_pct', 98.5)
        
        row = {
            # Scenario info
            'scenario_name': result['scenario_name'],
            'base_fee_bps': result.get('base_fee_bps', config.get('base_fee_bps', 30)),
            'market_condition': result.get('market_condition', 'normal'),
            'run_time_seconds': result.get('run_time_seconds', 0),
            
            # Fee configuration (standardized to percentages)
            'treasury_share_pct': treasury_pct,
            'creator_share_pct': creator_pct,
            'buffer_share_pct': buffer_pct,
            'sol_volatility_daily': config.get('sol_volatility_daily', 0.05),
            
            # Performance metrics
            'floor_growth_rate_annual': analysis['floor_growth_rate_annual'],
            'avg_floor_to_market_ratio': analysis.get('avg_floor_to_market_ratio', 0),
            'pomm_deployments': analysis['pomm_deployments'],
            'total_volume': analysis['total_volume'],
            'total_fees': analysis['total_fees'],
            'protocol_efficiency': analysis['protocol_efficiency'],
            'buffer_utilization': analysis.get('buffer_utilization', 0),
            'lp_yield_apy': analysis.get('lp_yield_apy', 0),
            
            # Final balances
            'final_treasury_balance': analysis['final_treasury_balance'],
            'final_buffer_balance': analysis['final_buffer_balance'],
            'final_deployed_feelssol': analysis['final_deployed_feelssol'],
            'buffer_routed_cumulative': analysis['buffer_routed_cumulative'],
            'mint_cumulative': analysis['mint_cumulative'],
            
            # Derived metrics
            'treasury_yield_pct': (analysis['final_treasury_balance'] / analysis['total_fees'] * 100) if analysis['total_fees'] > 0 else 0,
            'floor_advancement_rate': analysis['pomm_deployments'] / analysis['simulation_hours'] * 24 if analysis['pomm_deployments'] > 0 else 0,  # deployments per day
        }
        
        df_data.append(row)
    
    df = pd.DataFrame(df_data)
    
    print(f"Created DataFrame with {len(df)} rows and {len(df.columns)} columns")
    print(f"Scenarios: {df['scenario_name'].unique()}")
    print(f"Fee ranges: {sorted(df['base_fee_bps'].unique())} bps")
    print(f"Market conditions: {df['market_condition'].unique()}")
    
    # Display summary statistics
    print("\n=== SUMMARY STATISTICS ===")
    print(df[['floor_growth_rate_annual', 'final_treasury_balance', 'pomm_deployments', 'protocol_efficiency']].describe())
    
    # Note about data limitations
    if len(df) == 1:
        print("\nWARNING: Only 1 simulation result found.")
        print("   For comprehensive analysis, run parameter sweeps first:")
        print("   just sweep --sweep-type full --hours 168")
        print("   This notebook shows analysis framework with limited data.")
        
else:
    print("WARNING: No successful results to analyze. Please run parameter sweeps first.")
    print("Example commands:")
    print("  just sweep --sweep-type fee_range")
    print("  just sweep --sweep-type market_conditions") 
    print("  just sweep --sweep-type full --hours 168")

## 3. Fee Structure Performance Analysis

In [ ]:
# Analyze performance by fee scenario
if 'df' in locals():
    
    # Group by scenario and calculate mean performance
    scenario_performance = df.groupby('scenario_name').agg({
        'floor_growth_rate_annual': ['mean', 'std', 'min', 'max'],
        'final_treasury_balance': ['mean', 'std', 'min', 'max'],
        'pomm_deployments': ['mean', 'std', 'min', 'max'],
        'protocol_efficiency': ['mean', 'std', 'min', 'max'],
        'total_volume': ['mean', 'std'],
        'treasury_share_pct': 'first',
        'creator_share_pct': 'first',
        'buffer_share_pct': 'first'
    }).round(4)
    
    print("=== PERFORMANCE BY FEE SCENARIO ===")
    print(scenario_performance)
    
    # Create performance comparison visualization with centralized styling
    fig, axes = create_figure_with_style(2, 3, DEFAULT_STYLE.figure_size_detailed)
    if fig is not None:
        fig.suptitle('Fee Scenario Performance Comparison', fontsize=16, weight='bold', y=0.98)
        
        scenarios = df['scenario_name'].unique()
        colors = DEFAULT_STYLE.color_palette
        
        # Floor growth rate
        scenario_means = df.groupby('scenario_name')['floor_growth_rate_annual'].mean()
        bars1 = axes[0, 0].bar(scenarios, scenario_means * 100, 
                              color=colors[:len(scenarios)], alpha=DEFAULT_STYLE.alpha, 
                              edgecolor='white', linewidth=DEFAULT_STYLE.linewidth)
        axes[0, 0].set_title('Floor Growth Rate (Annual %)', weight='bold')
        axes[0, 0].tick_params(axis='x', rotation=45)
        axes[0, 0].set_ylabel('Annual Growth (%)')
        
        # Treasury accumulation
        treasury_means = df.groupby('scenario_name')['final_treasury_balance'].mean()
        bars2 = axes[0, 1].bar(scenarios, treasury_means, 
                              color=colors[:len(scenarios)], alpha=DEFAULT_STYLE.alpha, 
                              edgecolor='white', linewidth=DEFAULT_STYLE.linewidth)
        axes[0, 1].set_title('Treasury Balance (FeelsSOL)', weight='bold')
        axes[0, 1].tick_params(axis='x', rotation=45)
        axes[0, 1].set_ylabel('Treasury (FeelsSOL)')
        
        # POMM deployments
        pomm_means = df.groupby('scenario_name')['pomm_deployments'].mean()
        bars3 = axes[0, 2].bar(scenarios, pomm_means, 
                              color=colors[:len(scenarios)], alpha=DEFAULT_STYLE.alpha, 
                              edgecolor='white', linewidth=DEFAULT_STYLE.linewidth)
        axes[0, 2].set_title('POMM Deployments', weight='bold')
        axes[0, 2].tick_params(axis='x', rotation=45)
        axes[0, 2].set_ylabel('Deployments')
        
        # Protocol efficiency
        efficiency_means = df.groupby('scenario_name')['protocol_efficiency'].mean()
        bars4 = axes[1, 0].bar(scenarios, efficiency_means, 
                              color=colors[:len(scenarios)], alpha=DEFAULT_STYLE.alpha, 
                              edgecolor='white', linewidth=DEFAULT_STYLE.linewidth)
        axes[1, 0].set_title('Protocol Efficiency', weight='bold')
        axes[1, 0].tick_params(axis='x', rotation=45)
        axes[1, 0].set_ylabel('Efficiency (USD/FeelsSOL)')
        
        # Trading volume
        volume_means = df.groupby('scenario_name')['total_volume'].mean()
        bars5 = axes[1, 1].bar(scenarios, volume_means, 
                              color=colors[:len(scenarios)], alpha=DEFAULT_STYLE.alpha, 
                              edgecolor='white', linewidth=DEFAULT_STYLE.linewidth)
        axes[1, 1].set_title('Trading Volume (FeelsSOL)', weight='bold')
        axes[1, 1].tick_params(axis='x', rotation=45)
        axes[1, 1].set_ylabel('Volume (FeelsSOL)')
        
        # Buffer utilization
        buffer_util_means = df.groupby('scenario_name')['buffer_utilization'].mean()
        bars6 = axes[1, 2].bar(scenarios, buffer_util_means * 100, 
                              color=colors[:len(scenarios)], alpha=DEFAULT_STYLE.alpha, 
                              edgecolor='white', linewidth=DEFAULT_STYLE.linewidth)
        axes[1, 2].set_title('Buffer Utilization (%)', weight='bold')
        axes[1, 2].tick_params(axis='x', rotation=45)
        axes[1, 2].set_ylabel('Utilization (%)')
        
        # Adjust layout and styling
        import matplotlib.pyplot as plt
        plt.tight_layout()
        plt.subplots_adjust(top=0.93)
        plt.show()

## 4. Fee Level Sensitivity Analysis

In [ ]:
# Analyze how performance varies with fee levels
if 'df' in locals() and 'base_fee_bps' in df.columns:
    
    # Filter for fee range analysis (normal market conditions)
    fee_analysis_df = df[df['market_condition'] == 'normal'].copy() if 'market_condition' in df.columns else df.copy()
    
    print(f"Fee sensitivity analysis with {len(fee_analysis_df)} data points")
    
    if len(fee_analysis_df) > 0:
        # Check if we have multiple fee levels
        unique_fees = sorted(fee_analysis_df['base_fee_bps'].unique())
        scenarios_with_multiple_fees = []
        
        for scenario in fee_analysis_df['scenario_name'].unique():
            scenario_df = fee_analysis_df[fee_analysis_df['scenario_name'] == scenario]
            scenario_fees = sorted(scenario_df['base_fee_bps'].unique())
            if len(scenario_fees) > 1:
                scenarios_with_multiple_fees.append(scenario)
        
        print(f"Available fee levels: {unique_fees} bps")
        print(f"Scenarios with multiple fee levels: {scenarios_with_multiple_fees}")
        
        if len(unique_fees) > 1:
            # Create fee sensitivity plots with centralized styling
            fig, axes = create_figure_with_style(2, 2, DEFAULT_STYLE.figure_size_detailed)
            if fig is not None:
                fig.suptitle('Fee Level Sensitivity Analysis', fontsize=16, weight='bold', y=0.98)
                
                colors = DEFAULT_STYLE.color_palette
                
                # For each scenario, plot performance vs fee level
                for i, scenario in enumerate(fee_analysis_df['scenario_name'].unique()):
                    scenario_df = fee_analysis_df[fee_analysis_df['scenario_name'] == scenario].sort_values('base_fee_bps')
                    
                    if len(scenario_df) > 1:  # Only plot if we have multiple fee levels
                        color = colors[i % len(colors)]
                        
                        # Floor growth vs fee level
                        axes[0, 0].plot(scenario_df['base_fee_bps'], scenario_df['floor_growth_rate_annual'] * 100, 
                                       marker='o', label=scenario, alpha=DEFAULT_STYLE.alpha, 
                                       linewidth=DEFAULT_STYLE.linewidth, markersize=8, color=color, 
                                       markeredgecolor='white', markeredgewidth=DEFAULT_STYLE.marker_edgewidth)
                        
                        # Treasury balance vs fee level
                        axes[0, 1].plot(scenario_df['base_fee_bps'], scenario_df['final_treasury_balance'], 
                                       marker='s', label=scenario, alpha=DEFAULT_STYLE.alpha, 
                                       linewidth=DEFAULT_STYLE.linewidth, markersize=8, color=color, 
                                       markeredgecolor='white', markeredgewidth=DEFAULT_STYLE.marker_edgewidth)
                        
                        # Trading volume vs fee level
                        axes[1, 0].plot(scenario_df['base_fee_bps'], scenario_df['total_volume'], 
                                       marker='^', label=scenario, alpha=DEFAULT_STYLE.alpha, 
                                       linewidth=DEFAULT_STYLE.linewidth, markersize=8, color=color, 
                                       markeredgecolor='white', markeredgewidth=DEFAULT_STYLE.marker_edgewidth)
                        
                        # Protocol efficiency vs fee level
                        axes[1, 1].plot(scenario_df['base_fee_bps'], scenario_df['protocol_efficiency'], 
                                       marker='D', label=scenario, alpha=DEFAULT_STYLE.alpha, 
                                       linewidth=DEFAULT_STYLE.linewidth, markersize=8, color=color, 
                                       markeredgecolor='white', markeredgewidth=DEFAULT_STYLE.marker_edgewidth)
                
                # Configure plots with centralized styling
                axes[0, 0].set_title('Floor Growth Rate vs Fee Level', weight='bold')
                axes[0, 0].set_xlabel('Base Fee (bps)')
                axes[0, 0].set_ylabel('Annual Growth (%)')
                axes[0, 0].legend(bbox_to_anchor=(1.05, 1), loc='upper left', frameon=True, fancybox=True, shadow=True)
                
                axes[0, 1].set_title('Treasury Balance vs Fee Level', weight='bold')
                axes[0, 1].set_xlabel('Base Fee (bps)')
                axes[0, 1].set_ylabel('Treasury (FeelsSOL)')
                axes[0, 1].legend(bbox_to_anchor=(1.05, 1), loc='upper left', frameon=True, fancybox=True, shadow=True)
                
                axes[1, 0].set_title('Trading Volume vs Fee Level', weight='bold')
                axes[1, 0].set_xlabel('Base Fee (bps)')
                axes[1, 0].set_ylabel('Volume (FeelsSOL)')
                axes[1, 0].legend(bbox_to_anchor=(1.05, 1), loc='upper left', frameon=True, fancybox=True, shadow=True)
                
                axes[1, 1].set_title('Protocol Efficiency vs Fee Level', weight='bold')
                axes[1, 1].set_xlabel('Base Fee (bps)')
                axes[1, 1].set_ylabel('Efficiency (USD/FeelsSOL)')
                axes[1, 1].legend(bbox_to_anchor=(1.05, 1), loc='upper left', frameon=True, fancybox=True, shadow=True)
                
                import matplotlib.pyplot as plt
                plt.tight_layout()
                plt.subplots_adjust(top=0.93)
                plt.show()
            
            # Calculate fee elasticity
            print("\n=== FEE ELASTICITY ANALYSIS ===")
            
            # Overall fee elasticity across all scenarios
            overall_elasticity_data = []
            
            for scenario in scenarios_with_multiple_fees:
                scenario_df = fee_analysis_df[fee_analysis_df['scenario_name'] == scenario].sort_values('base_fee_bps')
                
                if len(scenario_df) >= 2:
                    # Calculate elasticity between lowest and highest fee
                    low_fee = scenario_df.iloc[0]
                    high_fee = scenario_df.iloc[-1]
                    
                    volume_change_pct = (high_fee['total_volume'] - low_fee['total_volume']) / low_fee['total_volume'] if low_fee['total_volume'] > 0 else 0
                    fee_change_pct = (high_fee['base_fee_bps'] - low_fee['base_fee_bps']) / low_fee['base_fee_bps'] if low_fee['base_fee_bps'] > 0 else 0
                    
                    elasticity = volume_change_pct / fee_change_pct if fee_change_pct != 0 else 0
                    
                    print(f"{scenario}:")
                    print(f"  Fee range: {low_fee['base_fee_bps']}-{high_fee['base_fee_bps']} bps")
                    print(f"  Volume change: {volume_change_pct:.2%}")
                    print(f"  Elasticity: {elasticity:.2f}")
                    print()
                    
                    overall_elasticity_data.append({
                        'scenario': scenario,
                        'elasticity': elasticity,
                        'volume_change': volume_change_pct
                    })
            
            # Calculate average elasticity
            if overall_elasticity_data:
                avg_elasticity = np.mean([d['elasticity'] for d in overall_elasticity_data])
                print(f"Average fee elasticity across scenarios: {avg_elasticity:.2f}")
                
                # Interpretation
                if avg_elasticity < -1:
                    print("→ Demand is elastic: Volume decreases significantly with fee increases")
                elif avg_elasticity < 0:
                    print("→ Demand is inelastic: Volume decreases modestly with fee increases")
                else:
                    print("→ Unusual elasticity: Volume may increase with fees (investigation needed)")
            
            # Create elasticity visualization with centralized styling
            if len(overall_elasticity_data) > 1:
                fig, (ax1, ax2) = create_figure_with_style(1, 2, (14, 6))
                if fig is not None:
                    fig.suptitle('Fee Elasticity Analysis', fontsize=16, weight='bold', y=0.98)
                    
                    scenarios = [d['scenario'] for d in overall_elasticity_data]
                    elasticities = [d['elasticity'] for d in overall_elasticity_data]
                    volume_changes = [d['volume_change'] * 100 for d in overall_elasticity_data]
                    colors = DEFAULT_STYLE.color_palette
                    
                    # Elasticity by scenario
                    bars1 = ax1.bar(scenarios, elasticities, alpha=DEFAULT_STYLE.alpha, 
                                   color=colors[:len(scenarios)], edgecolor='white', 
                                   linewidth=DEFAULT_STYLE.linewidth)
                    ax1.set_title('Fee Elasticity by Scenario', weight='bold')
                    ax1.set_ylabel('Elasticity')
                    ax1.tick_params(axis='x', rotation=45)
                    ax1.axhline(y=0, color=DEFAULT_STYLE.error_color, linestyle='--', alpha=0.7, linewidth=2)
                    ax1.axhline(y=-1, color=DEFAULT_STYLE.secondary_color, linestyle='--', alpha=0.7, linewidth=2, label='Elastic threshold')
                    ax1.legend()
                    
                    # Volume change by scenario
                    bars2 = ax2.bar(scenarios, volume_changes, alpha=DEFAULT_STYLE.alpha, 
                                   color=colors[:len(scenarios)], edgecolor='white', 
                                   linewidth=DEFAULT_STYLE.linewidth)
                    ax2.set_title('Volume Change by Scenario', weight='bold')
                    ax2.set_ylabel('Volume Change (%)')
                    ax2.tick_params(axis='x', rotation=45)
                    ax2.axhline(y=0, color=DEFAULT_STYLE.error_color, linestyle='--', alpha=0.7, linewidth=2)
                    
                    import matplotlib.pyplot as plt
                    plt.tight_layout()
                    plt.subplots_adjust(top=0.90)
                    plt.show()
                
        else:
            print("\nWARNING: Fee sensitivity analysis requires multiple fee levels per scenario")
            print("Current data structure:")
            fee_summary = fee_analysis_df.groupby(['scenario_name', 'base_fee_bps']).size().reset_index(name='count')
            print(fee_summary)
            
            # Show what we can analyze instead with centralized styling
            print("\nALTERNATIVE ANALYSIS - Fee Level Comparison Across Scenarios")
            
            # Create scenario comparison at each fee level
            fig, axes = create_figure_with_style(2, 2, DEFAULT_STYLE.figure_size_detailed)
            if fig is not None:
                fig.suptitle('Performance Comparison Across Scenarios by Fee Level', fontsize=16, weight='bold', y=0.98)
                
                colors = DEFAULT_STYLE.color_palette
                
                for fee_level in unique_fees:
                    fee_data = fee_analysis_df[fee_analysis_df['base_fee_bps'] == fee_level]
                    
                    if len(fee_data) > 1:
                        scenarios = fee_data['scenario_name']
                        
                        # Floor growth comparison
                        axes[0, 0].bar([f"{s}\n@{fee_level}bps" for s in scenarios], 
                                      fee_data['floor_growth_rate_annual'] * 100, 
                                      alpha=DEFAULT_STYLE.alpha, label=f'{fee_level} bps', 
                                      color=colors[:len(scenarios)], edgecolor='white', 
                                      linewidth=DEFAULT_STYLE.linewidth)
                        
                        # Treasury balance comparison  
                        axes[0, 1].bar([f"{s}\n@{fee_level}bps" for s in scenarios],
                                      fee_data['final_treasury_balance'],
                                      alpha=DEFAULT_STYLE.alpha, label=f'{fee_level} bps', 
                                      color=colors[:len(scenarios)], edgecolor='white', 
                                      linewidth=DEFAULT_STYLE.linewidth)
                        
                        # Volume comparison
                        axes[1, 0].bar([f"{s}\n@{fee_level}bps" for s in scenarios],
                                      fee_data['total_volume'],
                                      alpha=DEFAULT_STYLE.alpha, label=f'{fee_level} bps', 
                                      color=colors[:len(scenarios)], edgecolor='white', 
                                      linewidth=DEFAULT_STYLE.linewidth)
                        
                        # Efficiency comparison
                        axes[1, 1].bar([f"{s}\n@{fee_level}bps" for s in scenarios],
                                      fee_data['protocol_efficiency'],
                                      alpha=DEFAULT_STYLE.alpha, label=f'{fee_level} bps', 
                                      color=colors[:len(scenarios)], edgecolor='white', 
                                      linewidth=DEFAULT_STYLE.linewidth)
                
                axes[0, 0].set_title('Floor Growth Rate by Scenario & Fee', weight='bold')
                axes[0, 0].set_ylabel('Annual Growth (%)')
                axes[0, 0].tick_params(axis='x', rotation=45)
                
                axes[0, 1].set_title('Treasury Balance by Scenario & Fee', weight='bold')
                axes[0, 1].set_ylabel('Treasury (FeelsSOL)')
                axes[0, 1].tick_params(axis='x', rotation=45)
                
                axes[1, 0].set_title('Trading Volume by Scenario & Fee', weight='bold')
                axes[1, 0].set_ylabel('Volume (FeelsSOL)')
                axes[1, 0].tick_params(axis='x', rotation=45)
                
                axes[1, 1].set_title('Protocol Efficiency by Scenario & Fee', weight='bold')
                axes[1, 1].set_ylabel('Efficiency (USD/FeelsSOL)')
                axes[1, 1].tick_params(axis='x', rotation=45)
                
                import matplotlib.pyplot as plt
                plt.tight_layout()
                plt.subplots_adjust(top=0.93)
                plt.show()
            
            print("\nINSIGHTS FROM CROSS-SCENARIO COMPARISON:")
            for fee_level in unique_fees:
                fee_data = fee_analysis_df[fee_analysis_df['base_fee_bps'] == fee_level]
                if len(fee_data) > 1:
                    best_growth = fee_data.loc[fee_data['floor_growth_rate_annual'].idxmax()]
                    best_treasury = fee_data.loc[fee_data['final_treasury_balance'].idxmax()]
                    print(f"\nAt {fee_level} bps:")
                    print(f"  Best floor growth: {best_growth['scenario_name']} ({best_growth['floor_growth_rate_annual']:.2%})")
                    print(f"  Best treasury: {best_treasury['scenario_name']} ({best_treasury['final_treasury_balance']:.1f} FeelsSOL)")
    else:
        print("No data available for fee sensitivity analysis")
        print("The automatic parameter sweep should have generated this data.")
        print("Please check that the sweep completed successfully.")
else:
    print("DataFrame not available - please run the data loading section first")

## 5. Market Condition Impact Analysis

In [ ]:
# Analyze performance under different market conditions
if 'df' in locals() and 'market_condition' in df.columns:
    
    market_conditions = df['market_condition'].unique()
    
    if len(market_conditions) > 1:
        print("=== MARKET CONDITION IMPACT ANALYSIS ===")
        
        # Performance by market condition
        market_performance = df.groupby(['market_condition', 'scenario_name']).agg({
            'floor_growth_rate_annual': 'mean',
            'final_treasury_balance': 'mean',
            'pomm_deployments': 'mean',
            'total_volume': 'mean',
            'protocol_efficiency': 'mean'
        }).round(4)
        
        print(market_performance)
        
        # Create market condition comparison plots with centralized styling
        fig, axes = create_figure_with_style(2, 2, DEFAULT_STYLE.figure_size_detailed)
        if fig is not None:
            fig.suptitle('Performance Across Market Conditions', fontsize=16, weight='bold', y=0.98)
            
            scenarios = df['scenario_name'].unique()
            colors = DEFAULT_STYLE.color_palette
            
            metrics = [
                ('floor_growth_rate_annual', 'Floor Growth Rate (%)', lambda x: x * 100),
                ('final_treasury_balance', 'Treasury Balance', lambda x: x),
                ('pomm_deployments', 'POMM Deployments', lambda x: x),
                ('total_volume', 'Trading Volume', lambda x: x)
            ]
            
            for idx, (metric, title, transform) in enumerate(metrics):
                ax = axes[idx // 2, idx % 2]
                
                # Create grouped bar chart with centralized styling
                x = np.arange(len(market_conditions))
                width = 0.8 / len(scenarios)
                
                for i, scenario in enumerate(scenarios):
                    values = []
                    for condition in market_conditions:
                        condition_data = df[(df['market_condition'] == condition) & 
                                          (df['scenario_name'] == scenario)]
                        if len(condition_data) > 0:
                            values.append(transform(condition_data[metric].mean()))
                        else:
                            values.append(0)
                    
                    color = colors[i % len(colors)]
                    ax.bar(x + i * width, values, width, label=scenario, alpha=DEFAULT_STYLE.alpha, 
                          color=color, edgecolor='white', linewidth=DEFAULT_STYLE.linewidth)
                
                ax.set_title(title, weight='bold')
                ax.set_xlabel('Market Condition')
                ax.set_xticks(x + width * (len(scenarios) - 1) / 2)
                ax.set_xticklabels(market_conditions, rotation=45)
                ax.legend(frameon=True, fancybox=True, shadow=True)
            
            import matplotlib.pyplot as plt
            plt.tight_layout()
            plt.subplots_adjust(top=0.93)
            plt.show()
        
        # Market condition resilience analysis
        print("\n=== MARKET RESILIENCE ANALYSIS ===")
        
        for scenario in scenarios:
            scenario_data = df[df['scenario_name'] == scenario]
            
            print(f"\n{scenario}:")
            
            for condition in market_conditions:
                condition_data = scenario_data[scenario_data['market_condition'] == condition]
                
                if len(condition_data) > 0:
                    avg_growth = condition_data['floor_growth_rate_annual'].mean()
                    avg_volume = condition_data['total_volume'].mean()
                    
                    print(f"  {condition}: Growth {avg_growth:.2%}, Volume {avg_volume:,.0f}")
        
        # Create resilience heatmap with centralized styling
        print("\n=== RESILIENCE HEATMAP ===")
        
        # Create pivot table for heatmap
        heatmap_data = df.pivot_table(
            values='floor_growth_rate_annual', 
            index='scenario_name', 
            columns='market_condition', 
            aggfunc='mean'
        ) * 100  # Convert to percentage
        
        if not heatmap_data.empty:
            import matplotlib.pyplot as plt
            import seaborn as sns
            
            fig, ax = create_figure_with_style(1, 1, (12, 8))
            if fig is not None:
                # Create heatmap with centralized styling
                mask = heatmap_data.isnull()
                sns.heatmap(heatmap_data, 
                           annot=True, 
                           fmt='.1f', 
                           cmap='RdYlBu_r',
                           center=heatmap_data.mean().mean(),
                           square=True,
                           mask=mask,
                           cbar_kws={"shrink": .8, "label": "Annual Growth Rate (%)"},
                           linewidths=0.5,
                           linecolor='white',
                           ax=ax)
                
                ax.set_title('Floor Growth Rate Resilience Across Market Conditions', 
                           fontsize=14, weight='bold', pad=20)
                ax.set_xlabel('Market Condition', weight='bold')
                ax.set_ylabel('Fee Scenario', weight='bold')
                plt.setp(ax.get_xticklabels(), rotation=45)
                plt.setp(ax.get_yticklabels(), rotation=0)
                plt.tight_layout()
                plt.show()
            
            # Calculate resilience scores
            print("\n=== RESILIENCE SCORES ===")
            
            # Calculate coefficient of variation (CV) as resilience metric
            # Lower CV = more resilient across market conditions
            resilience_scores = {}
            
            for scenario in heatmap_data.index:
                scenario_values = heatmap_data.loc[scenario].dropna()
                if len(scenario_values) > 1:
                    mean_growth = scenario_values.mean()
                    std_growth = scenario_values.std()
                    cv = std_growth / mean_growth if mean_growth > 0 else float('inf')
                    resilience_scores[scenario] = {
                        'mean_growth': mean_growth,
                        'volatility': std_growth,
                        'cv': cv,
                        'resilience_rank': None
                    }
            
            # Rank by coefficient of variation (lower is better)
            sorted_scenarios = sorted(resilience_scores.keys(), 
                                    key=lambda x: resilience_scores[x]['cv'])
            
            for rank, scenario in enumerate(sorted_scenarios, 1):
                resilience_scores[scenario]['resilience_rank'] = rank
                
                print(f"{rank}. {scenario}:")
                print(f"   Mean growth: {resilience_scores[scenario]['mean_growth']:.1f}%")
                print(f"   Volatility: {resilience_scores[scenario]['volatility']:.1f}%")
                print(f"   CV (lower=better): {resilience_scores[scenario]['cv']:.2f}")
                
                if rank == 1:
                    print("   → MOST RESILIENT across market conditions")
                elif rank == len(sorted_scenarios):
                    print("   → LEAST RESILIENT across market conditions")
                print()
    else:
        print("No market condition variations found in the data.")
        print("Current market conditions available:", df.get('market_condition', ['normal']).unique())
        print("Run market condition sweeps for comparative analysis:")
        print("   just sweep --sweep-type market_conditions")

## 6. Optimal Parameter Identification

In [ ]:
# Multi-objective optimization analysis
if 'df' in locals():
    
    print("=== OPTIMAL PARAMETER IDENTIFICATION ===")
    
    # Define optimization objectives
    objectives = {
        'max_floor_growth': ('floor_growth_rate_annual', 'maximize'),
        'max_treasury': ('final_treasury_balance', 'maximize'),
        'max_efficiency': ('protocol_efficiency', 'maximize'),
        'max_volume': ('total_volume', 'maximize'),
        'min_fee_impact': ('base_fee_bps', 'minimize')  # Lower fees better for adoption
    }
    
    # Find optimal configurations for each objective
    optimal_configs = {}
    
    for obj_name, (metric, direction) in objectives.items():
        if direction == 'maximize':
            optimal_row = df.loc[df[metric].idxmax()]
        else:
            optimal_row = df.loc[df[metric].idxmin()]
        
        optimal_configs[obj_name] = {
            'scenario': optimal_row['scenario_name'],
            'base_fee_bps': optimal_row['base_fee_bps'],
            'market_condition': optimal_row.get('market_condition', 'normal'),
            'treasury_share': optimal_row['treasury_share_pct'],
            'creator_share': optimal_row['creator_share_pct'],
            'buffer_share': optimal_row['buffer_share_pct'],
            'metric_value': optimal_row[metric]
        }
    
    # Display optimal configurations
    for obj_name, config in optimal_configs.items():
        print(f"\n{obj_name.upper()}:")
        print(f"  Scenario: {config['scenario']}")
        print(f"  Fee split: {config['buffer_share']:.1f}% buffer, {config['treasury_share']:.1f}% treasury, {config['creator_share']:.1f}% creator")
        print(f"  Base fee: {config['base_fee_bps']} bps")
        print(f"  Market condition: {config['market_condition']}")
        print(f"  Metric value: {config['metric_value']:.6f}")
    
    # Pareto frontier analysis with centralized styling
    print("\n=== PARETO FRONTIER ANALYSIS ===")
    
    # Focus on key trade-off: floor growth vs treasury accumulation
    fig, ax = create_figure_with_style(1, 1, (14, 10))
    if fig is not None:
        # Color by scenario with centralized styling
        scenarios = df['scenario_name'].unique()
        colors = DEFAULT_STYLE.color_palette
        
        for i, scenario in enumerate(scenarios):
            scenario_df = df[df['scenario_name'] == scenario]
            color = colors[i % len(colors)]
            
            ax.scatter(scenario_df['final_treasury_balance'], 
                       scenario_df['floor_growth_rate_annual'] * 100,
                       c=color, label=scenario, alpha=DEFAULT_STYLE.alpha, s=80, 
                       edgecolors='white', linewidth=DEFAULT_STYLE.linewidth)
        
        ax.set_xlabel('Treasury Balance (FeelsSOL)', fontsize=12, weight='bold')
        ax.set_ylabel('Floor Growth Rate (Annual %)', fontsize=12, weight='bold')
        ax.set_title('Trade-off: Treasury Accumulation vs Floor Growth', 
                     fontsize=14, weight='bold', pad=20)
        ax.legend(frameon=True, fancybox=True, shadow=True, fontsize=10)
        
        # Add annotations for extreme points
        max_growth_idx = df['floor_growth_rate_annual'].idxmax()
        max_treasury_idx = df['final_treasury_balance'].idxmax()
        
        ax.annotate(f'Max Growth\n{df.loc[max_growth_idx, "scenario_name"]}', 
                    xy=(df.loc[max_growth_idx, 'final_treasury_balance'], 
                        df.loc[max_growth_idx, 'floor_growth_rate_annual'] * 100),
                    xytext=(10, 10), textcoords='offset points',
                    bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7),
                    arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))
        
        ax.annotate(f'Max Treasury\n{df.loc[max_treasury_idx, "scenario_name"]}', 
                    xy=(df.loc[max_treasury_idx, 'final_treasury_balance'], 
                        df.loc[max_treasury_idx, 'floor_growth_rate_annual'] * 100),
                    xytext=(10, -20), textcoords='offset points',
                    bbox=dict(boxstyle='round,pad=0.3', facecolor='lightblue', alpha=0.7),
                    arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))
        
        import matplotlib.pyplot as plt
        plt.tight_layout()
        plt.show()
    
    # Create multi-dimensional Pareto analysis with centralized styling
    print("\n=== MULTI-DIMENSIONAL ANALYSIS ===")
    
    fig, axes = create_figure_with_style(2, 2, DEFAULT_STYLE.figure_size_detailed)
    if fig is not None:
        fig.suptitle('Multi-Objective Trade-off Analysis', fontsize=16, weight='bold', y=0.98)
        
        colors = DEFAULT_STYLE.color_palette
        
        # Trade-off plots with centralized styling
        trade_offs = [
            ('final_treasury_balance', 'protocol_efficiency', 'Treasury vs Efficiency'),
            ('floor_growth_rate_annual', 'total_volume', 'Growth vs Volume'),
            ('final_treasury_balance', 'total_volume', 'Treasury vs Volume'),
            ('protocol_efficiency', 'pomm_deployments', 'Efficiency vs Deployments')
        ]
        
        for idx, (x_metric, y_metric, title) in enumerate(trade_offs):
            ax = axes[idx // 2, idx % 2]
            
            for i, scenario in enumerate(scenarios):
                scenario_df = df[df['scenario_name'] == scenario]
                color = colors[i % len(colors)]
                
                x_vals = scenario_df[x_metric]
                y_vals = scenario_df[y_metric] * 100 if y_metric == 'floor_growth_rate_annual' else scenario_df[y_metric]
                
                ax.scatter(x_vals, y_vals, c=color, label=scenario, alpha=DEFAULT_STYLE.alpha, s=60,
                          edgecolors='white', linewidth=DEFAULT_STYLE.linewidth)
            
            ax.set_xlabel(x_metric.replace('_', ' ').title(), weight='bold')
            ax.set_ylabel(y_metric.replace('_', ' ').title() + (' (%)' if y_metric == 'floor_growth_rate_annual' else ''), weight='bold')
            ax.set_title(title, weight='bold')
            
            if idx == 0:  # Only show legend on first subplot
                ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', 
                         frameon=True, fancybox=True, shadow=True)
        
        import matplotlib.pyplot as plt
        plt.tight_layout()
        plt.subplots_adjust(top=0.93)
        plt.show()
    
    # Calculate compromise solutions
    print("\n=== COMPROMISE SOLUTIONS ===")
    
    # Normalize metrics to [0,1] for multi-objective scoring
    df_norm = df.copy()
    
    # Key metrics to optimize (higher is better for all after normalization)
    key_metrics = ['floor_growth_rate_annual', 'final_treasury_balance', 'protocol_efficiency']
    
    for metric in key_metrics:
        df_norm[f'{metric}_norm'] = (df[metric] - df[metric].min()) / (df[metric].max() - df[metric].min())
    
    # Calculate weighted scores for different priorities
    priority_scenarios = {
        'Floor Growth Priority': {'floor_growth_rate_annual_norm': 0.6, 'final_treasury_balance_norm': 0.2, 'protocol_efficiency_norm': 0.2},
        'Treasury Priority': {'floor_growth_rate_annual_norm': 0.2, 'final_treasury_balance_norm': 0.6, 'protocol_efficiency_norm': 0.2},
        'Balanced': {'floor_growth_rate_annual_norm': 0.4, 'final_treasury_balance_norm': 0.3, 'protocol_efficiency_norm': 0.3}
    }
    
    for priority_name, weights in priority_scenarios.items():
        df_norm[f'score_{priority_name}'] = sum(df_norm[metric] * weight for metric, weight in weights.items())
        
        best_idx = df_norm[f'score_{priority_name}'].idxmax()
        best_config = df.loc[best_idx]
        
        print(f"\n{priority_name}:")
        print(f"  Scenario: {best_config['scenario_name']}")
        print(f"  Fee split: {best_config['buffer_share_pct']:.1f}% buffer, {best_config['treasury_share_pct']:.1f}% treasury, {best_config['creator_share_pct']:.1f}% creator")
        print(f"  Base fee: {best_config['base_fee_bps']} bps")
        print(f"  Floor growth: {best_config['floor_growth_rate_annual']:.2%}")
        print(f"  Treasury: {best_config['final_treasury_balance']:.1f} FeelsSOL")
        print(f"  Efficiency: {best_config['protocol_efficiency']:.6f}")
        print(f"  Score: {df_norm.loc[best_idx, f'score_{priority_name}']:.3f}")
    
    # Create priority scoring visualization with centralized styling
    print("\n=== PRIORITY SCORING VISUALIZATION ===")
    
    fig, ax = create_figure_with_style(1, 1, (14, 8))
    if fig is not None:
        priority_names = list(priority_scenarios.keys())
        x = np.arange(len(scenarios))
        width = 0.25
        colors = DEFAULT_STYLE.color_palette
        
        for i, priority in enumerate(priority_names):
            score_col = f'score_{priority}'
            scenario_scores = []
            
            for scenario in scenarios:
                scenario_data = df_norm[df_norm['scenario_name'] == scenario]
                if len(scenario_data) > 0:
                    scenario_scores.append(scenario_data[score_col].mean())
                else:
                    scenario_scores.append(0)
            
            color = colors[i % len(colors)]
            ax.bar(x + i * width, scenario_scores, width, label=priority, 
                   alpha=DEFAULT_STYLE.alpha, color=color, edgecolor='white', 
                   linewidth=DEFAULT_STYLE.linewidth)
        
        ax.set_xlabel('Fee Scenario', weight='bold')
        ax.set_ylabel('Normalized Score', weight='bold')
        ax.set_title('Multi-Objective Priority Scoring by Scenario', weight='bold', pad=20)
        ax.set_xticks(x + width)
        ax.set_xticklabels(scenarios, rotation=45)
        ax.legend(frameon=True, fancybox=True, shadow=True)
        
        import matplotlib.pyplot as plt
        plt.tight_layout()
        plt.show()

## 7. Generate Governance Recommendations

In [ ]:
# Generate comprehensive governance recommendations
if 'df' in locals():
    
    print("=== GOVERNANCE RECOMMENDATIONS ===")
    print()
    
    # Check available scenarios
    available_scenarios = df['scenario_name'].unique()
    print(f"Available scenarios: {list(available_scenarios)}")
    
    # Key findings - handle case where some scenarios might not exist
    current_default = df[df['scenario_name'] == 'current_default']
    protocol_sustainable = df[df['scenario_name'] == 'protocol_sustainable'] if 'protocol_sustainable' in available_scenarios else pd.DataFrame()
    
    if len(current_default) > 0:
        current_avg = current_default.mean()
        
        print("1. CURRENT SCENARIO ANALYSIS:")
        print(f"   Current Default:")
        print(f"     - Fee split: {current_avg['buffer_share_pct']:.1f}% buffer, {current_avg['treasury_share_pct']:.1f}% treasury, {current_avg['creator_share_pct']:.1f}% creator")
        print(f"     - Floor growth: {current_avg['floor_growth_rate_annual']:.2%} annually")
        print(f"     - Treasury accumulation: {current_avg['final_treasury_balance']:.1f} FeelsSOL")
        print(f"     - POMM deployments: {current_avg['pomm_deployments']:.0f}")
        print()
        
        if len(protocol_sustainable) > 0:
            sustainable_avg = protocol_sustainable.mean()
            
            print(f"   Protocol Sustainable comparison:")
            print(f"     - Fee split: {sustainable_avg['buffer_share_pct']:.1f}% buffer, {sustainable_avg['treasury_share_pct']:.1f}% treasury, {sustainable_avg['creator_share_pct']:.1f}% creator")
            print(f"     - Floor growth: {sustainable_avg['floor_growth_rate_annual']:.2%} annually")
            print(f"     - Treasury accumulation: {sustainable_avg['final_treasury_balance']:.1f} FeelsSOL")
            print(f"     - POMM deployments: {sustainable_avg['pomm_deployments']:.0f}")
            print()
            
            treasury_improvement = sustainable_avg['final_treasury_balance'] / current_avg['final_treasury_balance'] - 1
            floor_impact = sustainable_avg['floor_growth_rate_annual'] / current_avg['floor_growth_rate_annual'] - 1
            
            print(f"   Moving to Protocol Sustainable scenario provides:")
            print(f"     - {treasury_improvement:+.1%} treasury accumulation")
            print(f"     - {floor_impact:+.1%} floor growth rate")
            print()
        else:
            print("   Note: Protocol sustainable scenario not found in current data.")
            print("   Run parameter sweeps to compare multiple scenarios.")
            print()
    
    # Fee level recommendations
    if 'base_fee_bps' in df.columns and len(df['base_fee_bps'].unique()) > 1:
        fee_analysis = df.groupby('base_fee_bps').agg({
            'total_volume': 'mean',
            'floor_growth_rate_annual': 'mean',
            'final_treasury_balance': 'mean'
        })
        
        optimal_fee = fee_analysis['floor_growth_rate_annual'].idxmax()
        
        print(f"2. FEE LEVEL RECOMMENDATIONS:")
        print(f"   Optimal base fee for floor growth: {optimal_fee} bps")
        print(f"   Fee elasticity observed: Trading volume inversely correlated with fees")
        print(f"   Recommendation: Start with 30 bps, monitor volume impact")
        print()
    else:
        print("2. FEE LEVEL RECOMMENDATIONS:")
        print("   Current base fee: 30 bps")
        print("   Recommendation: Run fee sweep analysis for optimization")
        print("   Command: just sweep --sweep-type fee_range")
        print()
    
    # Market condition resilience
    if 'market_condition' in df.columns and len(df['market_condition'].unique()) > 1:
        print("3. MARKET RESILIENCE:")
        resilience = df.groupby(['scenario_name', 'market_condition'])['floor_growth_rate_annual'].mean().unstack()
        
        if 'bear_market' in resilience.columns and 'bull_market' in resilience.columns:
            for scenario in resilience.index:
                bear_performance = resilience.loc[scenario, 'bear_market']
                bull_performance = resilience.loc[scenario, 'bull_market']
                resilience_ratio = bear_performance / bull_performance
                
                print(f"   {scenario}: {resilience_ratio:.2f} bear/bull ratio")
        print()
    else:
        print("3. MARKET RESILIENCE:")
        print("   Limited market condition data available")
        print("   Recommendation: Run market condition sweeps")
        print("   Command: just sweep --sweep-type market_conditions")
        print()
    
    # Strategic recommendations based on current data
    print("4. STRATEGIC RECOMMENDATIONS:")
    print()
    print("   Based on current simulation data:")
    
    if len(df) > 0:
        current_data = df.iloc[0]  # Use first row as reference
        print(f"   Current fee allocation: {current_data['buffer_share_pct']:.1f}% buffer, {current_data['treasury_share_pct']:.1f}% treasury, {current_data['creator_share_pct']:.1f}% creator")
        print(f"   Current floor growth: {current_data['floor_growth_rate_annual']:.2%} annually")
        print()
    
    print("   RECOMMENDED PHASES:")
    print("   PHASE 1 - PROTOCOL LAUNCH (Months 1-3):")
    print("   - Fee split: 98.5% buffer, 1.0% treasury, 0.5% creator")
    print("   - Base fee: 30 bps")
    print("   - Rationale: Maximize floor advancement to establish market confidence")
    print()
    print("   PHASE 2 - GROWTH (Months 4-12):")
    print("   - Fee split: 90% buffer, 7% treasury, 3% creator")
    print("   - Base fee: Monitor and adjust based on volume (25-40 bps)")
    print("   - Rationale: Balance floor growth with treasury accumulation")
    print()
    print("   PHASE 3 - MATURITY (Year 2+):")
    print("   - Fee split: 85% buffer, 10% treasury, 5% creator")
    print("   - Base fee: Competitive market rate")
    print("   - Rationale: Sustainable long-term operations")
    print()
    print("5. NEXT STEPS FOR COMPREHENSIVE ANALYSIS:")
    print("   - Run parameter sweeps: just sweep --sweep-type full --hours 168")
    print("   - Compare multiple fee scenarios")
    print("   - Test different market conditions")
    print("   - Analyze fee elasticity across ranges")
    print("   - Generate Pareto frontier analysis")
    
    # Export recommendations with available data
    recommendations = {
        "timestamp": datetime.now().isoformat(),
        "analysis_summary": {
            "total_simulations": len(df),
            "scenarios_tested": list(df['scenario_name'].unique()),
            "fee_range_tested": f"{df['base_fee_bps'].min()}-{df['base_fee_bps'].max()} bps" if len(df) > 1 else f"{df['base_fee_bps'].iloc[0]} bps",
            "market_conditions": list(df.get('market_condition', ['normal']).unique()),
            "data_limitation": "Limited data - comprehensive parameter sweeps recommended"
        },
        "current_performance": {
            "floor_growth_annual": float(df['floor_growth_rate_annual'].mean()) if len(df) > 0 else 0,
            "treasury_balance": float(df['final_treasury_balance'].mean()) if len(df) > 0 else 0,
            "pomm_deployments": int(df['pomm_deployments'].sum()) if len(df) > 0 else 0
        },
        "phase_recommendations": {
            "phase_1_launch": {"buffer": 98.5, "treasury": 1.0, "creator": 0.5, "base_fee_bps": 30},
            "phase_2_growth": {"buffer": 90, "treasury": 7, "creator": 3, "base_fee_bps": 30},
            "phase_3_maturity": {"buffer": 85, "treasury": 10, "creator": 5, "base_fee_bps": 30}
        },
        "next_steps": [
            "Run comprehensive parameter sweeps",
            "Test multiple fee scenarios",
            "Analyze market condition impacts",
            "Generate multi-objective optimization"
        ]
    }
    
    # Save governance recommendations
    gov_export_path = project_root / "experiments" / "runs" / "governance_recommendations.json"
    with open(gov_export_path, 'w') as f:
        json.dump(recommendations, f, indent=2)
    
    print(f"\nGovernance recommendations exported to {gov_export_path}")
    print("Note: Run parameter sweeps for comprehensive analysis")

## Summary

This parameter sweep analysis provides:

1. **Automatic Data Generation** - Runs comprehensive parameter sweeps when insufficient data is detected
2. **Comprehensive Performance Comparison** - All fee scenarios across multiple market conditions
3. **Fee Elasticity Analysis** - Understanding of volume response to fee changes
4. **Multi-Objective Optimization** - Pareto frontier analysis for competing objectives
5. **Market Resilience Assessment** - Protocol performance under various market conditions
6. **Phased Implementation Strategy** - Practical roadmap for protocol launch and evolution

**Automated Sweep Coverage:**
- **6 fee scenarios**: default, protocol_sustainable, creator_incentive, balanced_growth, minimum_protocol, maximum_protocol
- **4 fee levels**: 20, 30, 40, 50 basis points
- **4 market conditions**: normal, bull_market, bear_market, high_volatility
- **96 total simulations** providing robust statistical analysis

**Key Takeaways:**
- Current defaults (98.5/1/0.5) maximize floor growth but limit treasury accumulation
- Protocol sustainable scenario provides better long-term treasury funding
- Fee level optimization shows trade-offs between volume and revenue
- Market condition analysis reveals protocol resilience across different environments

**Recommended Path Forward:**
1. **Launch Phase**: Start with floor-growth-optimized settings (98.5/1/0.5)
2. **Growth Phase**: Transition to balanced approach as protocol matures
3. **Maturity Phase**: Implement sustainable long-term fee structure
4. **Continuous Monitoring**: Use this notebook to re-analyze as conditions change

**Next Steps:**
- Review governance recommendations in the generated JSON file
- Monitor real-world performance against simulation predictions
- Re-run analysis periodically with updated market data
- Adjust parameters based on actual protocol performance